# INF265 - 2018-1 - Laboratorio 8

En el presente laboratorio se abordarán técnicas de Procesamiento de Lenguaje Natural con el fin de procesar la información previo a la generación de vectores que puedan ser usados por algoritmos de aprendizaje supervisado y entrenar modelos de clasificación de textos. Para ello se utilizará el siguiente conjunto de datos:

<br>**Noticias extraídas de diversas páginas web entre Marzo y Agosto del 2014** (https://archive.ics.uci.edu/ml/machine-learning-databases/00359/). Las categorías de noticias incluyen negocios, ciencia y tecnología, entretenimiento y salud.

### Contenido del Dataset
Del repositorio del dataset se tiene la siguiente información sobre sus atributos:
- **ID** : the numeric ID of the article
- **TITLE** : the headline of the article
- **URL** : the URL of the article
- **PUBLISHER** : the publisher of the article
- **CATEGORY** : the category of the news item; one of: -- b : business -- t : science and technology -- e : entertainment -- m : health
- **STORY** : alphanumeric ID of the news story that the article discusses
- **HOSTNAME** : hostname where the article was posted
- **TIMESTAMP** : approximate timestamp of the article's publication, given in Unix time (seconds since midnight on Jan 1, 1970)

## 1. Carga de los datos (1p)
Un dataset con contenido textual puede presentarse directamente como un CSV con diferentes columnas (como el caso mostrado). Para ello, una posibilidad es usar la librería **pandas**

In [3]:
import pandas as pd
header_names = ["id","title","url","publisher","category","story","hostname","timestamp"]
df = pd.read_csv("NewsAggregatorDataset/newsCorpora.csv", sep='\t', names = header_names, header=None, index_col=0)

In [4]:
df.sample(5)

,title,url,publisher,category,story,hostname,timestamp
id,,,,,,,
222544,Brad Pitt and Matthew McConaughey ran into eac...,http://www.realitytvworld.com/news/brad-pitt-a...,Reality TV World,e,dmImOGit97BTReMEp5ZNPz3pSMS_M,www.realitytvworld.com,1400531762603
194213,Kim Kardashian & Kanye West Might Be Forced to...,http://thestir.cafemom.com/entertainment/17217...,The Stir,e,dKpBJ9x0LO9dFOMvR0aNemATGECSM,thestir.cafemom.com,1399569369734
111238,Movie clock,http://www.timesunion.com/living/article/Movie...,Albany Times Union,e,dLo264xHVWUXcfMoeF7tU6iFGWAEM,www.timesunion.com,1397461215191
202979,Dave Franco thinks brother James' underwear se...,http://pagesix.com/2014/05/07/dave-franco-thin...,Page Six,e,dD-geJykcN5EPwMMF7Hlc-ODEM0hM,pagesix.com,1399702985870
29272,Turkey blocks Twitter over incriminating recor...,http://www.kearneyhub.com/news/national/turkey...,Kearney Hub,b,drWjpn6_UzSkmiM_9qWYJe3-LIGPM,www.kearneyhub.com,1395504330844


Observe el conjunto de datos cargado. Los campos con los que se debe trabajar son __*title*__ (contenido textual del titular de la noticia) y __*category*__ (tipo de noticia, es decir, la etiqueta objetivo para la clasificación textual)
<br>**a. Extraer las descripciones del campo *title* en un Numpy Array**
<br>**b. Extraer las etiquetas del campo *category* en un Numpy Array**
<br>Se sugiere el filtro `[nombre_campo]` y la función `.values` de los DataFrame de pandas

In [5]:
#COMPLETAR
import numpy as np

titles =np.array(df["title"])
labels =np.array(df["category"])
titles

array(['Fed official says weak data caused by weather, should not slow taper',
       "Fed's Charles Plosser sees high bar for change in pace of tapering",
       'US open: Stocks fall after Fed official hints at accelerated tapering',
       ...,
       'Child who swallowed battery to have reconstructive surgery at Cincinnati  ...',
       'Phoenix boy undergoes surgery to repair throat damage - WFSB 3 Connecticut',
       'Phoenix boy undergoes surgery to repair throat damage - CBS 3 Springfield  ...'],
      dtype=object)

## 2. Exploración (7p)
Puede realizarlo directamente en el pandas DataFrame o en los Numpy Array generados

#### a. Identifique la cantidad de noticias por cada categoría

In [8]:
#COMPLETAR
print(len(df.loc[df["category"]=="e"]))
print(len(df.loc[df["category"]=="b"]))
print(len(df.loc[df["category"]=="t"]))
print(len(df.loc[df["category"]=="m"]))


152469
115967
108344
45639


**b. Sin ningún tipo de pre-procesamiento, identifique la cantidad de tokens totales en los títulos, el tamaño del vocabulario, y los 10 tokens más frecuentes** (puede usar la función `most_common` de `collections.Counter`)

In [10]:
#COMPLETAR
from collections import Counter
a=[]
for i in range(len(titles[0:10000])):
    
    p = titles[i].split()
    a=a+p
    
c=Counter(a)
c.most_common(10)

[('to', 1954),
 ('...', 1337),
 ('in', 1184),
 ('for', 1016),
 ('of', 917),
 ('on', 866),
 ('the', 807),
 ('and', 743),
 ('The', 692),
 ('a', 506)]

**c. Identifique todos los caracteres no alfanuméricos de las descripciones de los títulos, guarde una lista sin repeticiones**
<br>Use expresiones regulares para extraer dichos caracteres

In [11]:
import re
text = " ".join(titles) #Se concatenan todos los títulos en la variable "text"

#COMPLETAR
c=re.findall(r"\W",text)
a=Counter(c)
caracteresRaros=list(a.keys())

**Antes de limpiar los caracteres no alfanuméricos, aprovecharemos las expresiones regulares para identificar ciertas construcciones:**
<br><br>**d. Calcule la cantidad de casos donde se hace referencia al posesivo en inglés ('s), por ejemplo: Jhon's**
<br>**e. Calcule la cantidad de casos donde aparecen valores porcentuales con el símbolo % (sin decimales), por ejemplo: 12%**
<br>**f. Calcule la cantidad de casos donde aparecen valores porcentuales con el término _percent_ (sin decimales), por ejemplo: 12 percent**
<br>**g. Combine e y f en una sola expresión regular**
<br>**h. Calcule la cantidad de casos donde aparecen fechas numéricas mes/día, por ejemplo: 9/11**
<br>**i. Identifique y guarde en una lista todas las siglas (asuma que sólo son mayúsculas sin puntos intermedios), por ejemplo: MTV**


In [12]:
#COMPLETAR (Use una celda diferente para cada expresion regular, e indique la letra del enunciado al que corresponde)
#(d)
d=Counter( re.findall(r"[A-Za-z]+'s",text) )
#(e)
e=Counter( re.findall(r"\d+[\b]*%\b",text) )
#(f)
f=Counter( re.findall(r"\d+[\b]*percent\b",text) )
#(g)
g=Counter( re.findall(r"(\d+[\b]*(%|percent)\b)",text) )
#(h)
h=Counter( re.findall(r"[01]\d[/][0-3]\d",text) )
h

Counter({'02/07': 1,
         '03/06': 2,
         '03/07': 2,
         '03/10': 1,
         '03/12': 1,
         '03/18': 2,
         '03/20': 8,
         '03/21': 1,
         '03/25': 2,
         '03/27': 1,
         '03/28': 1,
         '03/31': 3,
         '04/04': 1,
         '04/06': 1,
         '04/10': 1,
         '04/13': 1,
         '04/14': 2,
         '04/15': 1,
         '04/16': 2,
         '04/18': 1,
         '04/20': 11,
         '04/21': 1,
         '04/25': 1,
         '04/29': 1,
         '05/05': 4,
         '05/12': 1,
         '05/13': 1,
         '05/16': 1,
         '05/20': 25,
         '05/21': 1,
         '05/22': 1,
         '06/03': 1,
         '06/04': 1,
         '06/05': 2,
         '06/07': 4,
         '06/13': 1,
         '06/15': 1,
         '06/17': 1,
         '06/19': 1,
         '06/20': 14,
         '06/23': 1,
         '06/25': 1,
         '06/26': 2,
         '06/29': 1,
         '06/30': 1,
         '07/03': 1,
         '07/04': 1,
         '

In [13]:
#(i)
my_dicc = Counter( re.findall(r"[A-Z][A-Z]+",text) )
siglas = list(my_dicc.keys())

## 3. Preprocesamiento (8p)

Debe realizar los cambios sobre el Numpy Array de las descripciones (titles)

**a.1. Remueva los caracteres de puntuación (y otros símbolos) que considere pertinente eliminar. Se sugiere usar expresiones regulares y lo calculado en (2.c). También puede aprovechar la lista de** `string.punctuation`

In [14]:
import string
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [15]:
#COMPLETAR

text_noPunctuation=re.sub(r"[\W]"," ",text)

titles_alphanum = text_noPunctuation
c[0:1000]

[' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ',',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 "'",
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ':',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 "'",
 ' ',
 ' ',
 "'",
 ',',
 ' ',
 ' ',
 ' ',
 ' ',
 "'",
 ' ',
 ':',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ':',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 "'",
 ' ',
 ':',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 "'",
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 '.',
 '%',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ':',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 '-',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ':',
 ' ',
 ' ',
 "'",
 ' ',
 '-',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 '-',
 ' ',
 ' ',
 ' ',
 ' ',
 "'",
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 '.',
 '.',
 '.',
 ' ',
 ' ',
 ' ',
 ' ',
 ' '

#### a.2. A partir del resultado en (a.1), calcule nuevamente la cantidad de tokens, el tamaño de vocabulario y los 10 tokens más frecuentes

In [16]:
#COMPLETAR
Counter(text_noPunctuation.split()).most_common(10)

[('to', 83567),
 ('s', 63275),
 ('in', 57820),
 ('of', 49459),
 ('for', 43303),
 ('the', 39277),
 ('on', 32837),
 ('and', 32190),
 ('The', 30763),
 ('a', 22421)]

#### b.1. Convierta la lista de títulos a minúsculas

In [17]:
#Puede usar lower():
"HeLLo WoRlD".lower()

'hello world'

In [18]:
#COMPLETAR
titles_lowercase = text_noPunctuation.lower()

#### b.2. A partir del resultado en (b.1), calcule nuevamente la cantidad de tokens, el tamaño de vocabulario y los 10 tokens más frecuentes

In [19]:
#COMPLETAR
Counter(titles_lowercase.split()).most_common(10)

[('to', 98772),
 ('the', 70589),
 ('in', 69471),
 ('s', 67990),
 ('of', 58597),
 ('for', 52794),
 ('on', 40374),
 ('and', 38257),
 ('a', 32875),
 ('with', 26996)]

**c.1 ¿Y la lista de entidades/siglas? (ver 2.i)**
<br>**Reduzca a minúsculas los títulos con excepción de los tokens que se encuentren en la lista de siglas**
<br>Se sugiere dividir el texto en tokens (`.split()`) y transformar cada token a minusculas (`.lower()`) si es que el token no se encuentra en la lista de siglas (p.e. `if token not in siglas_entidades`)

In [21]:
#COMPLETAR
tokens_textNoPunctuation = text_noPunctuation.split()
respuesta =[]
for i in tokens_textNoPunctuation[0:10000]:
    if (i  not in siglas):
        i = i.lower()
    respuesta.append(i)
#titles_lowercase_withEntities = token.lower() for token in tokens_textNoPunctuation

#### c.2. A partir del resultado en (c.1), calcule nuevamente la cantidad de tokens, el tamaño de vocabulario y los 10 tokens más frecuentes

In [22]:
#COMPLETAR
my_dicc2 = Counter(respuesta)
tam_my_dicc2 = len(my_dicc2)
print(my_dicc2.most_common(10))
print(tam_my_dicc2)

[('to', 244), ('s', 240), ('on', 172), ('in', 163), ('titanfall', 160), ('for', 158), ('the', 116), ('china', 116), ('and', 103), ('chiquita', 95)]
2043


#### d.1. Los *stopwords* o palabras vacías son términos que por lo general no guardan mucha información relevante para el procesamiento de la información textual (si no se analiza a nivel contextual). Por ello, para este caso, se removerán los stopwords de los títulos. Indique la cantidad de stopwords que está removiendo

In [23]:
import nltk
nltk.download('stopwords')
stopwords_eng = nltk.corpus.stopwords.words('english')
print(stopwords_eng)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jorge\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where'

In [24]:
#Una forma de filtrar elementos de una lista y concatenar el resultado final
" ".join([token for token in "hello world ACC".split() if token not in ["hello"]])

'world ACC'

In [45]:
#COMPLETAR

titles_no_stopwords = " ".join([token for token in respuesta if token not in stopwords_eng])
len(respuesta) - len(titles_no_stopwords.split())


2237

#### d.2. A partir del resultado en (c.1), calcule nuevamente la cantidad de tokens, el tamaño de vocabulario y los 10 tokens más frecuentes

In [34]:
#COMPLETAR
#si se refiere a c.1
cantidad = Counter(respuesta)

#si se refiere a d.1
len(titles_no_stopwords.split())
my_dicc3 = Counter(titles_no_stopwords.split())

print("cant tokens: ",len(titles_no_stopwords.split()))
print("tamanho vocabulario: ",len(my_dicc3))

my_dicc3.most_common(10)

cant tokens:  7763
tamanho vocabulario:  1935


[('titanfall', 160),
 ('china', 116),
 ('chiquita', 95),
 ('stocks', 86),
 ('US', 83),
 ('banana', 80),
 ('fyffes', 76),
 ('one', 74),
 ('gox', 74),
 ('mt', 73)]

**e. Responda y explique brevemente: ¿Qué ha sucedido con el tamaño del vocabulario en cada una de las etapas? ¿Considera beneficioso que ocurra esa tendencia (para una tarea de minería de textos)?**

#RESPONDER

Ha ido disminuyendo, ya que se han quitado los datos que no aportan mucho al texto, y nos hemos quedado con las palabras más importantes. Si es beneficioso, ya que la palabras restantes son palabras clave, que si brindan informacion valiosa sobre el texto

## 4. _Stem_ y _Lemma_ (4 puntos)

**Genere dos versiones o copias del dataset (`titles_stem` y `titles_lemma`) a partir de lo procesado hasta el final de la parte 3**
<br>Puede consultar en el siguiente link: http://www.nltk.org/book/ch03.html

**a. Sobre la primera copia (`titles_stem`), aplique *Stemming* sobre todos los tokens, con excepción de las siglas/entidades**
<br>*Stemming* es el proceso de reducir un término a una raíz que puede ser común a otros términos, pero no necesariamente tiene un significado real. El objetivo de esta tarea deriva en la reducción del vocabulario para simplificar algunos procesos de búsqueda o reducir las dimensiones

In [37]:
porter = nltk.PorterStemmer()
print(porter.stem("houses"))
print(porter.stem("been"))
print(porter.stem("beatifully"))
print(porter.stem("hello"))
print(porter.stem("execution"))

hous
been
beati
hello
execut


In [46]:
#lo procesado al final de la parte b es el texto que no
import copy
copia1 = copy.deepcopy(titles_no_stopwords)
copia2 = copy.deepcopy(titles_no_stopwords)
titles_stem_split = []

spliteado = copia1.split()
for i in spliteado:
    titles_stem_split.append(porter.stem(i))

titles_stem = " ".join([token for token in titles_stem_split])
titles_stem

'fed offici say weak data caus weather slow taper fed charl plosser see high bar chang pace taper US open stock fall fed offici hint acceler taper fed risk fall behind curv charl plosser say fed plosser nasti weather curb job growth plosser fed may acceler taper pace fed plosser taper pace may slow fed plosser expect US unemploy fall 6 2 end 2014 US job growth last month hit weather fed presid charl plosser ecb unlik end sterilis smp purchas trader ecb unlik end steril smp purchas trader EU half bake bank union could work europ reach crunch point bank union ecb focu stronger euro drown ecb messag keep rate low EU aim deal tackl fail bank forex pound drop one month low euro noyer say strong euro creat unwarr econom pressur EU week ahead march 10 14 bank resolut transpar ukrain ecb member noyer open kind tool euro anxieti wane bund top treasuri spain debt ralli noyer say strong euro creat unwarr econom pressur 1 noyer say stronger euro creat unwarr pressur economi bad loan trigger key fe

**b. Sobre la segunda copia (`titles_lemma`), aplique *Lemmatization* sobre todos los tokens, con excepción de las siglas/entidades**
<br>*Lemmatizing* es el proceso de reducir un término a una forma base, el cual sí debe ser una entrada léxica válida para el lenguaje (es decir, debería tener un significado real).

In [41]:
# Para realizar la lematización, se requiere instalar la base de datos de WordNet
nltk.download('wordnet')
wnl = nltk.WordNetLemmatizer()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jorge\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


In [42]:
# Solo transformará aquellos términos que encuentre en la base de datos de WordNet
print(wnl.lemmatize("houses"))
print(wnl.lemmatize("living"))
print(wnl.lemmatize("beatifully"))
print(wnl.lemmatize("hello"))
print(wnl.lemmatize("compared"))

house
living
beatifully
hello
compared


In [43]:
#COMPLETAR
titles_lemma_split = []

spliteado = copia2.split()
for i in spliteado:
    if (i  not in siglas):
        titles_lemma_split.append(wnl.lemmatize(i))
    else:
        titles_lemma_split.append(i)
    
titles_lemma = " ".join([token for token in titles_lemma_split])
titles_lemma

'fed official say weak data caused weather slow taper fed charles plosser see high bar change pace tapering US open stock fall fed official hint accelerated tapering fed risk falling behind curve charles plosser say fed plosser nasty weather curbed job growth plosser fed may accelerate tapering pace fed plosser taper pace may slow fed plosser expects US unemployment fall 6 2 end 2014 US job growth last month hit weather fed president charles plosser ECB unlikely end sterilisation SMP purchase trader ECB unlikely end sterilization SMP purchase trader EU half baked bank union could work europe reach crunch point banking union ECB FOCUS stronger euro drowns ECB message keep rate low EU aim deal tackling failing bank forex pound drop one month low euro noyer say strong euro creates unwarranted economic pressure EU week ahead march 10 14 bank resolution transparency ukraine ECB member noyer open kind tool euro anxiety wane bunds top treasury spain debt rally noyer say strong euro creates un

**c. Calcule el tamaño de vocabulario y los 20 tokens más frecuentes para cada uno de los arreglos en (a) y (b).**

In [49]:
#COMPLETAR
my_dicc4 = Counter(titles_stem_split)

print("split: ")
print("cant tokens: ",len(titles_stem_split))
print("tamanho vocabulario: ",len(my_dicc4))

print("stem: ",my_dicc4.most_common(20))

my_dicc5 = Counter(titles_lemma_split)
print("lemma: ")
print("cant tokens: ",len(titles_lemma_split))
print("tamanho vocabulario: ",len(my_dicc5))

print(my_dicc5.most_common(20))

split: 
cant tokens:  7763
tamanho vocabulario:  1560
stem:  [('titanfal', 160), ('stock', 116), ('china', 116), ('chiquita', 95), ('US', 83), ('banana', 83), ('bitcoin', 80), ('fyff', 77), ('one', 75), ('gox', 75), ('mt', 73), ('file', 68), ('xbox', 67), ('market', 66), ('data', 65), ('bankruptci', 63), ('sbarro', 59), ('world', 57), ('ukrain', 52), ('american', 51)]
lemma: 
cant tokens:  7763
tamanho vocabulario:  1766
[('titanfall', 160), ('china', 116), ('stock', 110), ('chiquita', 95), ('US', 83), ('banana', 83), ('fyffes', 76), ('one', 75), ('gox', 74), ('mt', 73), ('xbox', 67), ('data', 65), ('bitcoin', 65), ('file', 65), ('bankruptcy', 63), ('market', 60), ('sbarro', 59), ('world', 56), ('ukraine', 52), ('american', 51)]


**d. Responda brevemente: ¿El vocabulario de `titles_stem` es menor al de `titles_lemma`? ¿Por qué considera que obtuvo dicho resultado?**

In [51]:
#Sí, porque el indicador stem cuenta trabaja con la "raiz" de una palabra, intentando agrupar palabras que potencial-
#mente dicen los mismo, mientras que el otro solamente quita los lemas, y faltarían depurar mejor.

## 5. Vectorización del texto (Laboratorio 9)

#### Convierta el texto del título de las noticias a vectores de características utlizando `CountVectorizer` y `TfidfVectorizer`

\* Las clases `CountVectorizer` y `TfidfVectorizer` se encuentran en la librería scikit-learn